In [1]:
import mxnet as mx
from mxnet import nd
from mxnet import gluon

C:\Users\weij\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
import sys
sys.path.insert(0, '..')
import gluonbook as gb
from mxnet import nd, gluon, init, autograd
from mxnet.gluon import nn

def batch_norm(X, gamma, beta, moving_mean, moving_var,
               eps, momentum):
    # 通过 autograd 来获取是不是在训练环境下。
    if not autograd.is_training():
        # 如果是在预测模式下，直接使用传入的移动平滑均值和方差。
        X_hat = (X - moving_mean) / nd.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        # 接在全连接层后情况，计算特征维上的均值和方差。
        if len(X.shape) == 2:
            mean = X.mean(axis=0)
            var = ((X - mean)**2).mean(axis=0)
        # 接在二维卷积层后的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持 X
        # 的形状以便后面可以正常的做广播运算。
        else:
            mean = X.mean(axis=(0,2,3), keepdims=True)
            var = ((X - mean)**2).mean(axis=(0,2,3), keepdims=True)
        # 训练模式下用当前的均值和方差做归一化。
        X_hat = (X - mean) / nd.sqrt(var + eps)
        # 更新移动平滑均值和方差。
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    # 拉升和偏移
    Y = gamma * X_hat + beta
    return (Y, moving_mean, moving_var)

C:\Users\weij\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class BatchNorm(nn.Block):
    def __init__(self, num_features, num_dims, **kwargs):
        super(BatchNorm, self).__init__(**kwargs)
        shape = (1,num_features) if num_dims == 2 else (1,num_features,1,1)
        # 参与求导和更新的模型参数，分别初始化成 0 和 1。
        self.beta = self.params.get('beta', shape=shape, init=init.Zero())
        self.gamma = self.params.get('gamma', shape=shape, init=init.One())
        # 不参与求导的模型参数。全在 CPU 上初始化成 0。
        self.moving_mean = nd.zeros(shape)
        self.moving_variance = nd.zeros(shape)
    def forward(self, X):
        # 如果 X 不在 CPU 上，将 moving_mean 和 moving_varience 复制到对应设备上。
        if self.moving_mean.context != X.context:
            self.moving_mean = self.moving_mean.copyto(X.context)
            self.moving_variance = self.moving_variance.copyto(X.context)
        # 保存更新过的 moving_mean 和 moving_var。
        Y, self.moving_mean, self.moving_variance = batch_norm(
            X, self.gamma.data(), self.beta.data(), self.moving_mean,
            self.moving_variance, eps=1e-5, momentum=0.9)
        return Y

In [3]:
net = nn.Sequential()
net.add(
    nn.Conv2D(6, kernel_size=5),
#     BatchNorm(6, num_dims=4),
    nn.Activation('sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(16, kernel_size=5),
#     BatchNorm(16, num_dims=4),
    nn.Activation('sigmoid'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120),
#     BatchNorm(120, num_dims=2),
    nn.Activation('sigmoid'),
    nn.Dense(84),
#     BatchNorm(84, num_dims=2),
    nn.Activation('sigmoid'),
    nn.Dense(10)
)
lr = 1.0
ctx = gb.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train_data, test_data = gb.load_data_fashion_mnist(batch_size=256)
gb.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=100)


training on gpu(0)
epoch 1, loss 2.3167, train acc 0.103, test acc 0.187, time 4.7 sec
epoch 2, loss 1.3419, train acc 0.477, test acc 0.653, time 2.0 sec
epoch 3, loss 0.8395, train acc 0.673, test acc 0.697, time 1.9 sec
epoch 4, loss 0.7004, train acc 0.724, test acc 0.738, time 2.0 sec
epoch 5, loss 0.6251, train acc 0.753, test acc 0.769, time 2.0 sec
epoch 6, loss 0.5747, train acc 0.772, test acc 0.793, time 2.0 sec
epoch 7, loss 0.5273, train acc 0.795, test acc 0.797, time 2.0 sec
epoch 8, loss 0.4979, train acc 0.808, test acc 0.822, time 2.0 sec
epoch 9, loss 0.4724, train acc 0.820, test acc 0.822, time 2.0 sec
epoch 10, loss 0.4538, train acc 0.827, test acc 0.831, time 2.0 sec
epoch 11, loss 0.4316, train acc 0.839, test acc 0.846, time 2.0 sec
epoch 12, loss 0.4180, train acc 0.844, test acc 0.853, time 2.0 sec
epoch 13, loss 0.4023, train acc 0.852, test acc 0.859, time 2.0 sec
epoch 14, loss 0.3872, train acc 0.857, test acc 0.827, time 2.0 sec
epoch 15, loss 0.3767, t

In [3]:
net = nn.Sequential()
net.add(
    nn.Conv2D(6, kernel_size=5),
#     BatchNorm(6, num_dims=4),
    nn.Activation('relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(16, kernel_size=5),
#     BatchNorm(16, num_dims=4),
    nn.Activation('relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120),
#     BatchNorm(120, num_dims=2),
    nn.Activation('relu'),
    nn.Dense(84),
#     BatchNorm(84, num_dims=2),
    nn.Activation('relu'),
    nn.Dense(10)
)
lr = 0.1
ctx = gb.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train_data, test_data = gb.load_data_fashion_mnist(batch_size=256)
gb.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=100)

training on gpu(0)
epoch 1, loss 1.0193, train acc 0.620, test acc 0.700, time 4.7 sec
epoch 2, loss 0.5965, train acc 0.778, test acc 0.793, time 2.1 sec
epoch 3, loss 0.5032, train acc 0.815, test acc 0.836, time 2.1 sec
epoch 4, loss 0.4474, train acc 0.836, test acc 0.851, time 2.1 sec
epoch 5, loss 0.4068, train acc 0.851, test acc 0.855, time 2.1 sec
epoch 6, loss 0.3826, train acc 0.859, test acc 0.858, time 2.1 sec
epoch 7, loss 0.3661, train acc 0.865, test acc 0.868, time 2.1 sec
epoch 8, loss 0.3512, train acc 0.871, test acc 0.879, time 2.1 sec
epoch 9, loss 0.3388, train acc 0.875, test acc 0.877, time 2.1 sec
epoch 10, loss 0.3251, train acc 0.880, test acc 0.879, time 2.1 sec
epoch 11, loss 0.3177, train acc 0.882, test acc 0.880, time 2.1 sec
epoch 12, loss 0.3084, train acc 0.885, test acc 0.882, time 2.1 sec
epoch 13, loss 0.3015, train acc 0.888, test acc 0.882, time 2.1 sec
epoch 14, loss 0.2964, train acc 0.891, test acc 0.887, time 2.1 sec
epoch 15, loss 0.2903, t

In [3]:
net = nn.Sequential()
net.add(
    nn.Conv2D(6, kernel_size=5),
    BatchNorm(6, num_dims=4),
    nn.Activation('relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(16, kernel_size=5),
    BatchNorm(16, num_dims=4),
    nn.Activation('relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120),
    BatchNorm(120, num_dims=2),
    nn.Activation('relu'),
    nn.Dense(84),
    BatchNorm(84, num_dims=2),
    nn.Activation('relu'),
    nn.Dense(10)
)
lr = 0.1
ctx = gb.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train_data, test_data = gb.load_data_fashion_mnist(batch_size=256)
gb.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=100)

training on gpu(0)
epoch 1, loss 0.5965, train acc 0.794, test acc 0.838, time 6.7 sec
epoch 2, loss 0.3792, train acc 0.866, test acc 0.869, time 4.0 sec
epoch 3, loss 0.3327, train acc 0.880, test acc 0.878, time 4.1 sec
epoch 4, loss 0.3074, train acc 0.888, test acc 0.875, time 4.0 sec
epoch 5, loss 0.2876, train acc 0.896, test acc 0.891, time 4.1 sec
epoch 6, loss 0.2731, train acc 0.900, test acc 0.874, time 4.1 sec
epoch 7, loss 0.2600, train acc 0.905, test acc 0.895, time 4.1 sec
epoch 8, loss 0.2504, train acc 0.908, test acc 0.867, time 4.0 sec
epoch 9, loss 0.2434, train acc 0.910, test acc 0.899, time 3.9 sec
epoch 10, loss 0.2327, train acc 0.914, test acc 0.891, time 4.0 sec
epoch 11, loss 0.2264, train acc 0.916, test acc 0.898, time 3.9 sec
epoch 12, loss 0.2194, train acc 0.919, test acc 0.891, time 3.9 sec
epoch 13, loss 0.2117, train acc 0.922, test acc 0.898, time 4.0 sec
epoch 14, loss 0.2068, train acc 0.924, test acc 0.825, time 3.9 sec
epoch 15, loss 0.2011, t

In [4]:
net = nn.Sequential()
net.add(
    nn.Conv2D(6, kernel_size=5),
    BatchNorm(6, num_dims=4),
    nn.Activation('relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(16, kernel_size=5),
    BatchNorm(16, num_dims=4),
    nn.Activation('relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Dense(120),
    BatchNorm(120, num_dims=2),
    nn.Activation('relu'),
    nn.Dense(84),
    BatchNorm(84, num_dims=2),
    nn.Activation('relu'),
    nn.Dense(10)
)
lr = 0.1
ctx = gb.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
loss = gluon.loss.SoftmaxCrossEntropyLoss()
train_data, test_data = gb.load_data_fashion_mnist(batch_size=256)
gb.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=100)

training on gpu(0)
epoch 1, loss 0.5607, train acc 0.806, test acc 0.852, time 4.7 sec
epoch 2, loss 0.3746, train acc 0.865, test acc 0.870, time 4.1 sec
epoch 3, loss 0.3338, train acc 0.879, test acc 0.877, time 4.0 sec
epoch 4, loss 0.3074, train acc 0.888, test acc 0.883, time 4.0 sec
epoch 5, loss 0.2917, train acc 0.894, test acc 0.867, time 4.0 sec
epoch 6, loss 0.2783, train acc 0.898, test acc 0.886, time 4.0 sec
epoch 7, loss 0.2675, train acc 0.902, test acc 0.888, time 4.0 sec
epoch 8, loss 0.2586, train acc 0.906, test acc 0.893, time 4.0 sec
epoch 9, loss 0.2501, train acc 0.908, test acc 0.889, time 4.0 sec
epoch 10, loss 0.2430, train acc 0.910, test acc 0.895, time 3.9 sec
epoch 11, loss 0.2354, train acc 0.914, test acc 0.905, time 4.0 sec
epoch 12, loss 0.2301, train acc 0.915, test acc 0.897, time 4.0 sec
epoch 13, loss 0.2230, train acc 0.919, test acc 0.896, time 4.0 sec
epoch 14, loss 0.2177, train acc 0.920, test acc 0.897, time 4.0 sec
epoch 15, loss 0.2117, t